<a href="https://colab.research.google.com/github/mjgpinheiro/PyDBD/blob/master/DBD_Engine1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install plasmapy
import numpy as np
import matplotlib.pyplot as plt
from plasmapy.physics import Plasma, plasma_factory
from plasmapy.formulary import electric_field as ef
from plasmapy.formulary import ionization_state as ion_state
from plasmapy.atomic import particle_mass, particle_charge

# Define parameters
length = 1.0   # length of discharge (m)
width = 0.1    # width of discharge (m)
height = 0.01  # height of discharge (m)
freq = 60.0    # frequency of discharge (Hz)
voltage = 1000.0  # voltage of discharge (V)

# Calculate gas properties
gas = 'air'
mass = particle_mass(gas)
Z = particle_charge(gas)
T_e = 10.0  # electron temperature (eV)
T_g = 300.0  # gas temperature (K)
n_e = ion_state(gas, T_e=T_e, T_g=T_g, n=n)

# Retrieve ion species properties
ion_species = ion_state(gas)
n_i = plasma_factory(
    n_e=n_e,
    T_i=T_e,
    ion_species=ion_species
).ion_density

# Calculate characteristic plasma parameters
lambda_D = ef.Debye_length(T_e, n_e)
omega_pi = ef.plasma_frequency(n_e, particle=mass)
nu_i = ef.ion_collision_rate(n_e, ion_species, T_g)
nu_e = ef.electron_collision_rate(n_e, gas, T_e)

# Print results
print(f'Debye length: {lambda_D:.4e} m')
print(f'Plasma frequency: {omega_pi:.4e} Hz')
print(f'Ion collision rate: {nu_i:.4e} Hz')
print(f'Electron collision rate: {nu_e:.4e} Hz')

# Define simulation grid
x = np.linspace(0, length, 100)
y = np.linspace(0, height, 10)
z = np.linspace(0, width, 10)
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

# Calculate electric field
E = voltage / length

# Define Plasma instance
plasma = plasma_factory(
    n_e=n_e,
    T_e=T_e,
    ion_species=ion_species,
    V=None,
    E=E,
    L=length,
    W=width,
    H=height,
    uniform=False,
)

# Calculate current density
J = plasma.current_density

# Calculate air acceleration
a = J / plasma.rho

# Plot results
fig, ax = plt.subplots()
ax.pcolormesh(X, Y, a[:, :, 0])
ax.set_xlabel('Position (m)')
ax.set_ylabel('Height (m)')
ax.set_title('Air acceleration')
plt.show()

